Importing necessary libraries

In [1]:
from newspaper import Article
import pickle
from multiprocessing import Pool
from tqdm import tqdm
import numpy as np
import requests
import random
from time import time
import pandas as pd


data_path = '/Users/ruoyangzhang/Documents/PythonWorkingDirectory/news_exploration/Data/'

### loading in the article links

In [2]:
with open(data_path + 'article_links.pickle', 'rb') as handle:
    article_links = pickle.load(handle)

it seems as though there are many broken links (404 errors), we wish to estimate how many there are using a random sampling and then using the requests package to test if we receive a good response

In [3]:
def check_response(link):
    if not requests.get(link).ok:
        return(1)
    else:
        return(0)

In [157]:
sample_idx = random.sample(range(len(article_links)),500)

In [158]:
p = Pool(7, maxtasksperchild = 1)
test_res = p.map(check_response, tqdm([article_links[i] for i in sample_idx]))
p.close()


100%|██████████| 500/500 [00:00<00:00, 63540.43it/s]

In [160]:
print('the estimated percentage of broken links is:' ,100*(sum(test_res)/500), 'percent')

the estimated percentage of broken links is: 32.0 percent


## downloading the articles
32% seems acceptable

Let's try downloading the first 1k articles in the list to see if we get blocked and to time the entire process to estimate how long it would take for the entire dataset

In [4]:
def download_article_to_dict(link):
    if requests.get(link).ok:
        dict_to_return = {}
        article = Article(link)
        article.download()
        article.parse()
        dict_to_return['author'] = article.authors
        dict_to_return['title'] = article.title
        dict_to_return['publish_date'] = article.publish_date
        dict_to_return['text'] = article.text
        dict_to_return['images_url'] = ' '.join(list(article.images))
        dict_to_return['description'] = article.meta_description
        dict_to_return['lang'] = article.meta_lang
        return(dict_to_return)

In [5]:
begin = time()
p = Pool(8, maxtasksperchild=1)
download_res = p.map(download_article_to_dict, tqdm(article_links[:5000]))
p.close()
end = time()
time_lapsed = end - begin
print(time_lapsed)

100%|██████████| 5000/5000 [01:49<00:00, 45.64it/s]


ArticleException: Article `download()` failed with 404 Client Error: Not Found for url: https://www.reuters.com/article/us-brazil-museum-idUSKBN0TJ24D20151130 on URL http://www.reuters.com/article/us-brazil-museum-idUSKBN0TJ24D20151130

In [6]:
download_res

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [195]:
article = Article(link)
article.download()
article.parse()

In [11]:
print(article_links[0])

http://www.reuters.com/article/fbn-ravens-browns-writethru-idUSMTZEBC1NXSP3H20151201


In [6]:
article = Article(article_links[0])

In [7]:
article.download()

In [8]:
article.parse()

ArticleException: Article `download()` failed with 404 Client Error: Not Found for url: http://www.reuters.com/article/fbn-ravens-browns-writethru-idUSMTZEBC1NXSP3H20151201 on URL http://www.reuters.com/article/fbn-ravens-browns-writethru-idUSMTZEBC1NXSP3H20151201

In [28]:
requests.get(article_links[0])

<Response [404]>

We are blocked, we may be only able to download about 40K articles a day if we only use one machine with one ip address (no proxy)

In [31]:
fake = pd.read_csv(data_path + 'fake.csv')

In [34]:
fake.text[45]

'\n21st Century Wire says… \nMexico’s billionaire tycoon Carlos Slim saw a large chuck of his wealth evaporate – literally overnight, after news of a Donald Trump election victory hit the currency and stock markets on Wednesday morning. \nAccording to Bloomberg financial analysts, Slim, Mexico’s wealthiest person and rated fifth on the global rich list, took a stinging $5.1 billion haircut after the Mexican Peso went into a 12% free fall after Trump’s stunning upset over Democratic Hillary Clinton. \nSlim’s loss amounted to a staggering 9.2 percent of his total fortune. \nIn the week before, polls showed Hillary Clinton as the odds on favorite, netting the billionaires a staggering $57 billion in total wealth increase across world markets. However, according to the Bloomberg Billionaires Index, oligarchs lost a total of $41 billion during Wednesday’s epic slide – with Slim being the biggest loser. '